# 📌 Uygulama – 7  
# 🎥 Gerçek Zamanlı Webcam Inference (ImprovedCNN + CIFAR10 Normalize)

Bu uygulama ile artık:

> **Eğittiğim CIFAR10 tabanlı ImprovedCNN modelini,  
> webcam’den gelen canlı görüntü akışı üzerinde gerçek zamanlı çalıştırdım.**

Yani:

- Kameradan frame alıyorum  
- Her frame’i CIFAR10 formatına göre preprocess ediyorum  
- LibTorch ile modeli çalıştırıp sınıf tahmini yapıyorum  
- Sonucu anlık olarak görüntünün üzerine yazıyorum  

Bu, LibTorch + OpenCV tarafında **ilk tam gerçek zamanlı inference projem**.

---

## 🎯 1) Amaç — *Bu projede ne yaptım?*

Bu uygulamanın temel amacı:

> **Webcam’den gelen her kareyi (frame), CIFAR10 normalize pipeline’ından geçirip  
> ImprovedCNN TorchScript modeli ile sınıflandırmak ve sonucu ekranda göstermek.**

Artık:

- Tek görsel değil → Sürekli akış  
- Tek tahmin değil → Her frame için tahmin  
- Terminal çıktı değil → Görüntü üzerine yazı bindirme

---

## 🧩 2) Ana Adımlar — *Kodda yaptığım işlemler*

---

### 🟦 A) CIFAR10 sınıf isimlerini tanımladım

```cpp
const std::vector<std::string> CIFAR10_CLASSES = {
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
};
```
* Modelin verdiği sınıf index’i → label string’e çevirmek için kullanıyorum.

### 🟩 B) CIFAR10 normalize değerlerini tanımladım
```cpp
const float CIFAR10_MEAN[3] = {0.4914f, 0.4822f, 0.4465f};
const float CIFAR10_STD[3]  = {0.2470f, 0.2435f, 0.2616f};
```


Bu değerler:

* PyTorch CIFAR-10 dataset’inde kullanılan standart mean/std değerleridir.

* Eğitimde kullanılan normalize ile birebir aynı olmalıdır.




### 🟧 C) preprocess_image() – Webcam frame → Tensor

Aynı mantık: Uygulama 6’daki preprocess’in “webcam frame” versiyonu.

#### 1️⃣ Orijinal frame’i kopyaladım
```cpp
img_bgr.copyTo(img);
```

#### 2️⃣ Resize (64×64)
```cpp
cv::resize(img, img, cv::Size(target_size, target_size));
```

#### 3️⃣ BGR → RGB
```cpp
cv::cvtColor(img, img, cv::COLOR_BGR2RGB);
```

#### 4️⃣ uint8 → float32 ve 0–1 normalize
```cpp
img.convertTo(img_float, CV_32F, 1.0f / 255.0f);
```

#### 5️⃣ Kanal bazlı normalize (R, G, B ayrı ayrı)
```cpp
std::vector<cv::Mat> channels(3);
cv::split(img_float, channels);

for (int c = 0; c < 3; ++c)
{
    channels[c] = (channels[c] - CIFAR10_MEAN[c]) / CIFAR10_STD[c];
}

cv::merge(channels, img_float);
```


Bu sayede:

* Her kanal için: (pixel - mean) / std

* Python’daki transform ile birebir aynı pipeline.

### 6️⃣ HWC → Tensor (NHWC)
```cpp
auto tensor_image = torch::from_blob(
    img_float.data,
    {1, img_float.rows, img_float.cols, 3},
    torch::TensorOptions().dtype(torch::kFloat32)
);
```

#### 7️⃣ NHWC → NCHW + clone()
```cpp
tensor_image = tensor_image.permute({0, 3, 1, 2});
tensor_image = tensor_image.contiguous().clone();
```


Artık elimde:

* [1, 3, 64, 64] formatında

* float32, normalize edilmiş

* LibTorch’un kendi belleğinde duran güvenli bir tensör var.

### 🟥 D) run_inference() – Modüler forward fonksiyonu
```cpp
std::vector<torch::jit::IValue> inputs;
inputs.push_back(input);

torch::Tensor output = module.forward(inputs).toTensor();
```


* Kameradan gelen her frame için bu fonksiyonu kullanıyorum.

* Model ile preprocess ayrılmış durumda, kod yapısı temiz.

### 🟦 E) main() – Gerçek zamanlı döngü
#### ✔ 1) Modeli yükledim
```cpp
torch::jit::script::Module module = torch::jit::load(model_path);
module.to(torch::kCPU);
module.eval();
```

#### ✔ 2) Webcam’i açtım
```cpp
cv::VideoCapture cap(0);
if (!cap.isOpened()) { ... }
```


0 = Varsayılan kamera (laptop/webcam).

#### ✔ 3) Sonsuz döngüde frame okumaya başladım
```cpp
while (true)
{
    cv::Mat frame;
    cap >> frame;

    if (frame.empty()) { ... continue; }
    ...
}


### 🟩 F) Her frame için yaptığım işlemler

#### Preprocess
```cpp
torch::Tensor input_tensor = preprocess_image(frame, target_size);
```


#### Inference
```cpp
torch::Tensor output = run_inference(module, input_tensor);
```


#### Softmax + argmax
```cpp
torch::Tensor probs = torch::softmax(output, 1);
torch::Tensor pred_class = probs.argmax(1);
int pred_idx = pred_class.item<int>();
```


#### Index → sınıf ismi

```cpp
std::string pred_name = "unknown";
if (pred_idx >= 0 && pred_idx < CIFAR10_CLASSES.size())
    pred_name = CIFAR10_CLASSES[pred_idx];
```


#### Sonucu görüntü üzerine yazdım
```cpp
std::string text = "Pred: " + pred_name + " (idx=" + std::to_string(pred_idx) + ")";
cv::putText(frame, text,
            cv::Point(10, 30),
            cv::FONT_HERSHEY_SIMPLEX,
            0.8,
            cv::Scalar(0, 255, 0),
            2);
```


#### Frame’i gösterdim
```cpp
cv::imshow("Real-time CIFAR10 Inference", frame);
```


#### Çıkış tuşu (q veya ESC) kontrolü
```cpp
char key = static_cast<char>(cv::waitKey(1));
if (key == 'q' || key == 27) break;
```


---
## 🧠 3) Bu projede öğrendiğim temel kavramlar

📌 Gerçek zamanlı video akışını işlemek (cv::VideoCapture, cap >> frame)

📌 Her frame için preprocess + inference + postprocess pipeline’ı kurmak

📌 CIFAR10 normalize’nin canlı görüntüye uygulanması

📌 Model çıktısını sınıf index’inden etiket string’ine çevirmek

📌 cv::putText ile tahmini görüntü üzerine yazmak

📌 cv::waitKey(1) ile canlı pencere döngüsü yönetmek

📌 Çıkış için klavye tuşları ile kontrol (q, ESC)

📌 Webcam + LibTorch + OpenCV üçlüsünü birlikte kullanmak

Bu proje ile artık:

**Statik resim inference → Gerçek zamanlı canlı inference
seviyesine geçmiş oldum.**


---
## ⚠️ 4) Dikkat edilmesi gereken noktalar

model_ts.pt exe ile aynı klasörde olmalı

Kameranın 0 index’i doğru aygıt olmalı (VideoCapture(0))

Preprocess hızının FPS’i etkileyeceğini bilmeliyim

Normalize değerleri Python tarafıyla birebir aynı olmak zorunda

**Eğer GPU kullanılırsa:**

* Model ve tensörler aynı device’ta olmalı

* waitKey(1) çok büyük verilirse görüntü akıcılığı bozulur
---

## 🔖 5) Etiketler / Kategori

Real-Time Inference

Webcam + LibTorch

CIFAR10 Normalize

ImprovedCNN

OpenCV Video

Production-Style Loop

---

## 📝 6) Tek Cümlelik Özet

Eğittiğim ImprovedCNN CIFAR-10 modelini, C++ + LibTorch + OpenCV kullanarak webcam üzerinden gerçek zamanlı olarak çalıştırdım ve her karede tahmin edilen sınıfı görüntü üzerine yazdırdım.